# Projeto II - Processamento de Linguagem Natural

**Nome**: Guilherme Ferreira Galdino <br>
**RA**  : 11201811063 

## Descrição

Chatbot criado para obter informações de diversos jogos, como review, feedbacks, recomendações e dúvidas em geral.

<img src='https://first-bucket-course-alura.s3.us-east-2.amazonaws.com/fluxo.png'>

1. O Chatbot recupera a frase do usuario e tenta descobrir qual o contexto/topico que se trata.
2. Nos topicos, existem 5 possibilidades: 
<ul>
  <li>2.1 Saudações</li>
  <li>2.2 Review/Resumo</li>
  <li>2.3 Feedback</li>
  <li>2.4 Recomendacao</li>
  <li>2.5 Perguntas e respostas</li>
</ul>
3. Em saudações irá reconhecer os padrões utilizando unigramas e bigramas. Caso corresponda com aqueles previamente inseridos, irá responder o cumprimento com as opções de reposta de forma aleatória.
4. Review/Resumo
<ul>
<li>4.1 o chatbot tentará reconhecer o nome do jogo</li>
<li>4.2 obterá o nome do jogo mais próximo da base do kaggle</li>
<li>4.3 irá fazer a raspagem da review do jogo recuperado no site gamespot</li>
<li>4.4 executará a sumarização da review</li>
<li>4.5 devido o site ser em inglês, será feito a tradução para o português</li>
<li>4.6 por fim, mostra a resposta gerada no processo</li>
</ul>
5. Feedback
<ul>
<li>5.1 o chatbot tentará reconhecer o nome do jogo</li>
<li>5.2 obterá o nome do jogo mais próximo da base do kaggle</li>
<li>5.3 irá fazer a raspagem no mesmo site do gamespot, porém obtendos os comentários dos usuários na plataforma</li>
<li>5.4 une todos os comentários e realiza a sumarização, para não ultrapassar o limite que o modelo de anlise de sentimento estabelece</li>
<li>5.5 aplica o modelo de analise sentimento para obter se os comentarios são positivos, neutros ou negativos</li>
<li>5.6 retorna a resposta gerada no processo</li>
</ul>
6. Recomendação
<ul>
<li>6.1 o chatbot tentará reconhecer o nome do jogo</li>
<li>6.2 obterá o nome do jogo mais próximo da base do kaggle</li>
<li>6.3 com a mesma base do kaggle, obterá informações dos outros jogos sobre o gênero e a plataforma</li>
<li>6.4 fará a comparação por similaridade do jogo mencionado com os restantes da base</li>
<li>6.5 retorna os 5 jogos mais semelhantes com aquele mencionado</li>
</ul>
7. Perguntas e respostas
<ul>
<li>7.1 o chatbot tentará reconhecer o nome do jogo</li>
<li>7.2 obterá o nome do jogo mais próximo da base do kaggle</li>
<li>7.3 realiza uma busca no site do wikipedia sobre o determinado jogo</li>
<li>7.4 realiza a vetorização de todos os textos presentes na página</li>
<li>7.5 encontra as similaridades das frases com a pergunta feita</li>
<li>7.6 retorna a resposta que se aproximou mais com a pergunta</li>
</ul>

## Dependencias

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!spacy download pt_core_news_sm

2022-08-28 20:24:48.619085: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 12.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import random
import re
import nltk
import string
from collections import Counter 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd 
import numpy as np
from transformers import pipeline 
from urllib.parse import quote_plus
import urllib
from bs4 import BeautifulSoup
import spacy 
import googletrans
import csv
import json 

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')
stopwords = nltk.corpus.stopwords.words('english')
stopwords_pt = spacy.lang.pt.stop_words.STOP_WORDS

sentiment_task = pipeline("sentiment-analysis")

URL_WIKIPEDIA = 'https://pt.wikipedia.org'
URL_GAMESPOT = 'https://www.gamespot.com'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


## PLN

### Pre processamento

In [ ]:
def remove_stop_words(tokens, language = 'en'):
    stop_words = stopwords
    if language == 'pt':
        stop_words = stopwords_pt

    new_text = []
    for token in tokens:
        if token not in stop_words and token not in string.punctuation:
            new_text.append(token)

    return new_text

In [ ]:
def pre_processing(text, stopwords = 'en', lemma = True):
    text = text.lower()

    text = re.sub(r"https?://[A-Za-z0-9./]+", " ", text)

    text = re.sub(r" +", ' ', text)

    tokens = []
    for token in nltk.word_tokenize(text):
        tokens.append(token)

    new_text = remove_stop_words(tokens, stopwords)

    text = ' '.join([str(element) for element in new_text])

    if lemma:
        doc = nlp(text)
        list = []
        for token in doc:
            list.append(token.lemma_)
        
        text = ' '.join([str(element) for element in list])

    return text 

### Sumarizacao

In [ ]:
def score_words(freq_dist):
    max_freq = max(freq_dist.values())

    for word in freq_dist.keys():
        freq_dist[word] = (freq_dist[word]/max_freq)
    
    return freq_dist

In [ ]:
def score_sentences(sents, freq_dist, max_len=30):
    sent_scores = {}

    for sent in sents:
        words = sent.split(' ')
        for word in words:
            if word.lower() not in freq_dist.keys() or len(words) >= max_len:
                continue

            if sent in sent_scores.keys():
                sent_scores[sent] += freq_dist[word.lower()]
            else:
                sent_scores[sent] = freq_dist[word.lower()]
    
    return sent_scores

In [ ]:
def summarization(text, k):
    clean_text = re.sub('\n', '', text)
    clean_text = re.sub('-', '', clean_text)
    clean_text = pre_processing(text)

    word_tkn = nltk.word_tokenize(clean_text)
    freq_dist = nltk.FreqDist(word_tkn)
    freq_dist = score_words(freq_dist)

    sent_tkn = nltk.sent_tokenize(text)
    sent_scores = score_sentences(sent_tkn, freq_dist)

    top_sents = Counter(sent_scores)
    tops = top_sents.most_common(k)

    summary = ''
    for top in tops:
        summary += top[0].strip() + ' '

    return summary[:-1]

### Similaridade

In [ ]:
def get_similarity(sents_pre_process):
    tfidf = TfidfVectorizer()
    words_vectors = tfidf.fit_transform(sents_pre_process)

    similarity = cosine_similarity(words_vectors[-1], words_vectors)

    return similarity

In [ ]:
def get_most_similar(content, text):
    sents = nltk.sent_tokenize(content)

    sents_pre_process = []
    for i in range(len(sents)):
        sents_pre_process.append(pre_processing(sents[i], stopwords='pt', lemma=True))

    text = pre_processing(text, stopwords='pt', lemma=True)
    sents_pre_process.append(text)

    similarity = get_similarity(sents_pre_process)

    sent_idx = similarity.argsort()[0][-2]
    similar_vector = similarity.flatten()
    similar_vector.sort()
    vector_found = similar_vector[-2]

    if vector_found != 0:
        return sents[sent_idx]
    
    return None 

In [ ]:
def get_model_similarity(doc):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
    tfidf_matrix = tf.fit_transform(doc)
    doc_sim = cosine_similarity(tfidf_matrix)
    doc_sim_df = pd.DataFrame(doc_sim)

    return doc_sim_df

In [ ]:
def most_games_similarities(game_name, games, doc_sims, top = 5):
    game_idx = np.where(games == game_name.lower())[0][0]
    game_similarities = doc_sims.iloc[game_idx].values 
    similar_game_idxs = np.argsort(-game_similarities)[1:top + 1]
    similar_games = games[similar_game_idxs]

    return similar_games

### Analise de Sentimento

In [ ]:
def get_label_sentiment_by_score(score):
    label = ''
    if score > 0.8:
        label = 'muito boa'
    elif score > 0.4:
        label = 'boa'
    elif score > 0:
        label = 'razoavel'
    elif score > -0.4:
        label = 'ruim'
    elif score > -0.7:
        label = 'pessima'
    else:
        label = 'horrivel'

    return label 

In [ ]:
def sentiment_analysis(text):
    text = summarization(text, 512)

    sents = nltk.sent_tokenize(text)
    sentiments = sentiment_task(sents)

    score = 0
    count = 0
    for sentiment in sentiments:
        label = sentiment['label'].lower()
        if label == 'positive':
            score += sentiment['score']
        else:
            score -= sentiment['score']
        count += 1
    
    score = score / count 

    return get_label_sentiment_by_score(score)

### Perguntas e repostas

In [ ]:
def get_wiki_content_by_topic(topic):
    safe_string = quote_plus(topic)

    link = f'https://pt.wikipedia.org/w/index.php?search={safe_string}&title=Especial:Pesquisar&profile=advanced&fulltext=1&searchengineselect=mediawiki&ns0=1'
    
    dados = urllib.request.urlopen(link)
    dados_html = BeautifulSoup(dados, 'lxml')

    ul = dados_html.select('#mw-content-text > div.searchresults > ul')[0]
    li = ul.find_all('li')[0]
    href = li.find('a').attrs['href']

    link = URL_WIKIPEDIA + href 
    dados = urllib.request.urlopen(link)
    dados_html = BeautifulSoup(dados, 'lxml')
    paragraphs = dados_html.find_all('p')
    content = ''
    for p in paragraphs:
        content += p.text 
    content = content.lower()

    return content

In [ ]:
def question_answer(text, topic):
    content = get_wiki_content_by_topic(topic)
    
    return get_most_similar(content, text)

### Modelo de Linguagem com N-gramas

In [ ]:
def n_gram_extractor(sentence, n):
    grams = []
    tokens = re.sub(r'([^\s\w]|_)+', ' ', sentence).split()
    for i in range(len(tokens)-n+1):
        g = ' '.join(tokens[i:i+n])
        grams.append(g)

    return grams 

In [ ]:
def n_gram_extractor_range(sentence, start, end):
    grams = []
    if start == end:
        return n_gram_extractor(sentence, start)

    for i in range(start, end + 1):
        g_n = n_gram_extractor(sentence, i)
        grams.extend(g_n)

    return grams 

### Traducao

In [ ]:
def translate_sentence(sentence, source = 'en', destination = 'pt'):
    translator = googletrans.Translator()

    result = translator.translate(sentence, src=source, dest=destination)

    return result.text

## Jogos

### Review

In [ ]:
def getSoup(link):
    response = urllib.request.urlopen(link)
    html_doc = response.read()
    with open('index.html', 'w') as f:
        f.write(str(html_doc))
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    return soup

In [ ]:
def getLinkReview(name):
    safe_string = quote_plus(name)
    url = URL_GAMESPOT + f'/search/?q={safe_string}&i=reviews'
    soup = getSoup(url)

    list = soup.select('#js-sort-filter-results')[0]
    game = list.find('li')
    link_review = game.find('a')

    return URL_GAMESPOT + link_review.get('href')

In [ ]:
def getReview(name):
    link_review = getLinkReview(name)
    soup = getSoup(link_review)

    article = soup.find('article')
    content = article.select('div.js-content-entity-body')[0]
    paragraphs = content.find_all('p')
    
    review = ''
    for paragraph in paragraphs:
        review += paragraph.getText()
    
    return review 

### Feedback

In [ ]:
def get_item_by_class(list, class_name):
    result = ''
    for item in list:
        try:
            classes = item.attrs['class']
            cls = ' '.join(classes)
            if class_name in cls:
                result = item
                break 
        except:
            result = ''
    
    return result 

In [ ]:
def get_item_by_attr_contains(list, attr, value):
    result = ''
    for item in list:
        try:
            atr = item.attrs[attr]
            if value in atr:
                result = atr
                break 
        except:
            result = ''

    return result 

In [ ]:
def get_list_item_cotains(list, attr, value):
    result = []
    count = 0
    for item in list:
        try:
            atr = item.attrs[attr]
            if value in atr:
                result.append(item)
        except:
            count += 1

    return result 

In [ ]:
def get_html_comments(soup):
    article = soup.find('article')
    links = article.find_all('a')
    comment_id = get_item_by_attr_contains(links, 'href', 'comments')
    
    match = re.findall(r'[0-9]', comment_id)
    id = ''.join(match)

    link_comments = URL_GAMESPOT  + f'/forums/comments/{id}/?subTopic=0&comment_page=1&wrap=1'
    response = urllib.request.urlopen(link_comments)
    response_json = response.read()
    response_json = response_json.decode('ascii')
    response_json = json.loads(response_json)
    
    return response_json['html']

In [ ]:
def get_paragraphs_comments(div_comments):
    result = []
    for div in div_comments:
        article = div.find('article')
        paragraphs = article.find_all('p')
        for p in paragraphs:
            result.append(p.getText())
    
    return result 

In [ ]:
def getFeedbacks(name):
    link_review = getLinkReview(name)
    soup = getSoup(link_review)
    html = get_html_comments(soup)
    soup = BeautifulSoup(html, 'html.parser')

    divs = soup.find_all('div')
    div_comments = get_item_by_class(divs, 'comment-messages')
    div_comments = div_comments.find_all('div')
    div_comments = get_list_item_cotains(div_comments, 'id', 'js-message')
    comments = get_paragraphs_comments(div_comments)

    return comments 

### Nomes/Info jogos

In [ ]:
def get_games_names():
    file = open('/content/video_game_dataset.csv')
    csvReader = csv.reader(file)
    data = list(csvReader)

    names = []
    for item in data:
        names.append(item[1])

    return names 

In [ ]:
def get_games_names_df():
    df = pd.read_csv('/content/video_game_dataset.csv')

    names = df['Name'].values
    lower = lambda x: x.lower()
    vfunc = np.vectorize(lower)
    return vfunc(names)


In [ ]:
def get_info_games():
    df = pd.read_csv('/content/video_game_dataset.csv')

    df['info'] = df['Platform'] + ' ' + df['Genre']
    info = list(df['info'])

    return info

### Reconhecimento Nomes Jogos

In [ ]:
def get_most_similar_game(words, value):
    score = 0
    idx = 0
    for i, word in enumerate(words):
        sents = word.split(' ')
        sents.append(value)
        similarity = get_similarity(sents)
        aux = similarity[0].sum()

        if aux > score:
            idx = i 
            score = aux 

    return (words[idx], score)

In [ ]:
def named_game_recognition(text):
    text = pre_processing(text)
    tokens = nltk.word_tokenize(text)
    tokens = remove_stop_words(tokens, 'pt')

    text = ' '.join(tokens)
    tokens = n_gram_extractor_range(text, 1, 2)
    
    games = get_games_names()
 
    max_score = 0
    game_name = ''
    for token in tokens:
        name, score = get_most_similar_game(games, token)
        if score > max_score:
            max_score = score
            game_name = name 
    
    if max_score <= 1:
        return None 

    return game_name

In [ ]:
def getGameName(text):
    name = named_game_recognition(text)
    
    if name is None:
        return (False, 'Nao reconheci o nome do jogo. Pode repetir o nome dele ?')
    
    return (True, name)

## Topicos

### Saudacoes

In [ ]:
greetings_entry = ('olá', 'opa', 'oi', 'ola', 'eae', 'de boa', 'tudo certo', 'tudo bom', 'bom dia', 'boa tarde', 'boa noite', 'oba')
greetings_response = ('olá', 'opa', 'oi', 'bem-vindo', 'como você está?', 'tudo bem?')

In [ ]:
def checkGreetings(text):
    grams = n_gram_extractor_range(text, 1, 2)

    for gram in grams:
        if gram.lower() in greetings_entry:
            return True 

    return False

In [ ]:
def greetings(text):
    grams = n_gram_extractor_range(text, 1, 2)

    for gram in grams:
        if gram.lower() in greetings_entry:
            return random.choice(greetings_response) 

    return 'Nao entendi, pode repetir ?'

### Recomendacao

In [ ]:
def checkRecommendation(input):
    options = ['recomendacao', 'recomendacoes', 'semelhantes', 'semelhante']

    for option in options:
        if option in input:
            return True 
    
    return False  

In [ ]:
def recommendation(input):
    try:
        success, result = getGameName(input)
        if not success:
            update_active_topic('recommendation')
            return result 

        game_name = result 
        print('recuperando recomendacoes semelhantes ao jogo: ' + game_name)

        info = get_info_games()
        model = get_model_similarity(info)
        games = get_games_names_df()
        similar_games = most_games_similarities(game_name, games, model)
        
        update_active_topic('')
        return 'Jogos semelhantes: ' + ', '.join(similar_games)
    except Exception as e:
        print(str(e))
        return 'Nao foi possivel obter a recomendacao'

### Feedback

In [ ]:
def checkFeedback(input):
    options = ['feedback', 'opinioes', 'percepcao']

    for option in options:
        if option in input:
            return True 
    
    return False 

In [ ]:
def feedback(input):
    try:
        success, result = getGameName(input)
        if not success:
            update_active_topic('feedback')
            return result 

        game_name = result 
        print('buscando os feedbacks do jogo: ' + game_name)
        feedbacks = getFeedbacks(game_name)
        
        text = ' '.join(feedbacks)
        result = sentiment_analysis(text)

        update_active_topic('')
        return 'opiniao ' + result + ' do jogo'
    except Exception as e:
        print(str(e))
        return 'Nao foi possivel obter os feedbacks do jogo'

### Review

In [ ]:
def checkReview(input):
    options = ['review', 'quero saber', 'informacoes', 'informacao']

    for option in options:
        if option in input:
            return True 
    
    return False  

In [ ]:
def review(input):
    try:
        success, result = getGameName(input)
        if not success:
            update_active_topic('review')
            return result 

        gameName = result 
        print('recuperando a review do jogo: ' + gameName)
        review = getReview(gameName)

        short_review = summarization(review, 5)

        short_review = translate_sentence(short_review)

        update_active_topic('')
        return short_review
    except Exception as e:
        print(str(e))
        return 'Nao foi possivel obter a review'

### Perguntas e respostas

In [ ]:
def checkQuestionAnswer(input):
    return '?' in input

In [ ]:
def questionAnswer(input):
    try:
        success, result = getGameName(input)
        if not success:
            update_active_topic('question_answer')
            update_input(input)
            return result 

        old_input = get_active_input()
        if old_input != '':
            input = old_input

        game_name = result 
        print('encontrando a resposta da pergunta = ' + input)
        answer = question_answer(input, game_name)

        answer = re.sub(r"\[[0-9]*\]", '', answer)
        answer = answer.strip()

        update_input('')
        if answer is None:
            return 'Nao sei responder a pergunta :('

        return answer 
    except Exception as e:
        print(str(e))
        return 'Nao foi possivel obter a resposta da pergunta'

### Topicos

In [ ]:
topics = { 
    'greetings' : (checkGreetings, greetings), 
    'recommendation' : (checkRecommendation, recommendation),
    'feedback' : (checkFeedback, feedback),
    'review' : (checkReview, review),
    'question_answer' : (checkQuestionAnswer, questionAnswer),
    'active' : '',
    'input' : ''
}


In [ ]:
def get_topic(input):
    active_topic = get_active_topic()
    if active_topic != '':
        return active_topic

    for key, value in topics.items():
        if isinstance(value, str):
            continue

        check = value[0]
        if check(input):
            return key
        
    return None 

In [ ]:
def get_response(input, topic):
    response = topics[topic][1]
    return response(input)

## Chatbot

In [ ]:
def output(input, topic):
    if topic is None:
        return 'Desculpe, mas nao entendi. Pode repetir ?'

    response = get_response(input, topic)

    return response 


In [ ]:
def run():
    print("(escreva 'sair' para finalizar)")
    print("Ola, sou um chatbot e vou responder perguntas sobre jogos: ")

    while True:
        user_input = input('Usuario: ')
        user_input = user_input.lower()

        if user_input == 'sair':
            print('Ate breve!')
            break 
        
        topic = get_topic(user_input)
        response = output(user_input, topic)
        print('Chatbot:', response)
        

In [ ]:
run()

(escreva 'sair' para finalizar)
Ola, sou um chatbot e vou responder perguntas sobre jogos: 
Usuario: oi
Chatbot: como você está?
Usuario: bom dia
Chatbot: opa
Usuario: quero saber sobre o fifa
recuperando a review do jogo: FIFA 16
Chatbot: As inclinações e fraquezas do ano passado - ritmo, bolas exageradas, comportamento enlouquecedor do defensor - foram tomadas com severidade, na medida em que jogar FIFA 16 parece aprender um novo jogo. A postura do caranguejo que protege a bola não oferece mais uma defesa impenetrável; é mais fácil contornar os jogadores com a posse da bola e depois colocar um pé nela. Os tackles de slide recuperaram um pouco da antiga potência; os jogadores realmente deslizam novamente, o que significa que é possível ganhar a bola de uma distância inesperada. O Draft tira o prazer de construir uma equipe - reunindo uma química forte, a emoção de abrir pacotes - e oferece isso a você sem a necessidade de separar sua equipe principal. Custa 15.000 moedas (ou 300 micro